Stochastic Military Port Selection Problem
==========================================

In [ ]:
# Title: Stochastic Military Port Selection Problem
# Description: This extends the 
# Author: Lennon Conson

Load Packages
--------------

In [ ]:
import pyomo.environ as pyo